In [ ]:
import pyaudio
import wave
import requests
import json
import os
from dotenv import load_dotenv
import numpy as np
import time
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# 환경변수 불러오기
load_dotenv()

# Watson API Credential
STT_API_KEY = os.getenv('STT_API_KEY')
STT_URL = os.getenv('STT_URL')
TTS_API_KEY = os.getenv('TTS_API_KEY')
TTS_URL = os.getenv('TTS_URL')

# LLM Credential
API_KEY = os.getenv('API_KEY')
PROJECT_ID = os.getenv('PROJECT_ID')
IBM_CLOUD_URL = os.getenv('IBM_CLOUD_URL')
MODEL_ID = os.getenv('MODEL_ID')

# 오디오 설정
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
SILENCE_THRESHOLD = 1000
SILENCE_DURATION = 2

# LLM 세팅
generate_params = {GenParams.MAX_NEW_TOKENS: 100}
model = Model(
    model_id=MODEL_ID,
    params=generate_params,
    credentials={"apikey": API_KEY, "url": IBM_CLOUD_URL},
    project_id=PROJECT_ID
)

def record_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    print("녹음 중입니다... 말을 해주세요.")

    frames = []
    silent_chunks = 0

    while True:
        data = stream.read(CHUNK)
        frames.append(data)
        audio_data = np.frombuffer(data, dtype=np.int16)
        amplitude = np.abs(audio_data).mean()
        if amplitude < SILENCE_THRESHOLD:
            silent_chunks += 1
        else:
            silent_chunks = 0
        if silent_chunks > (SILENCE_DURATION * RATE / CHUNK):
            break

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open("input.wav", 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    return "input.wav"

def korean_speech_to_text(audio_path):
    endpoint = f"{STT_URL}/v1/recognize"
    headers = {"Content-Type": "audio/wav"}
    auth = ("apikey", STT_API_KEY)
    
    params = {
        'model': 'ko-KR_Multimedia',
        'timestamps': True,
        'word_confidence': True,
        'smart_formatting': True
    }

    try:
        with open(audio_path, 'rb') as audio_file:
            response = requests.post(endpoint, headers=headers, data=audio_file, params=params, auth=auth)

        if response.status_code == 200:
            result = response.json()
            
            if 'results' in result and len(result['results']) > 0:
                best_result = result['results'][0]['alternatives'][0]
                transcript = best_result['transcript'].strip()
                confidence = best_result.get('confidence', 0)
                
                print(f"인식된 텍스트: {transcript}")
                print(f"정확도: {confidence:.2%}")
                
                return transcript
            else:
                print("음성이 명확하게 인식되지 않았습니다.")
                return "(음성 인식 결과 없음)"
        else:
            print(f"음성 인식 오류: {response.status_code}")
            print(response.text)
            return "(음성 인식 오류)"
            
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {audio_path}")
        return "(파일 오류)"
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return "(STT 오류)"

def generate_response(text):
    system_prompt = "당신은 도움이 되는 한국어 비서입니다."
    formatted_prompt = f"<<SYS>>\n{system_prompt.strip()}\n<</SYS>>\n\n[INST]{text.strip()}[/INST]"
    try:
        response = model.generate(prompt=formatted_prompt)["results"][0]["generated_text"].strip()
    except Exception as e:
        print("LLM 오류:", str(e))
        response = "(LLM 오류)"
    return response

def synthesize_korean_text(text, filename="output.wav"):
    endpoint = f"{TTS_URL}/v1/synthesize"
    headers = {
        "Content-Type": "application/json",
        "Accept": "audio/wav"
    }
    payload = {"text": text}
    auth = ("apikey", TTS_API_KEY)
    params = {"voice": "ko-KR_JinV3Voice"}

    response = requests.post(endpoint, headers=headers, params=params, json=payload, auth=auth, stream=True)
    
    if response.status_code == 200:
        with open(filename, "wb") as audio_file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    audio_file.write(chunk)
        print(f"{filename} 파일이 저장되었습니다.")
        return filename
    else:
        print(f"음성 합성 오류: {response.status_code}")
        print(response.text)
        return None

def play_audio(audio_file):
    try:
        wf = wave.open(audio_file, 'rb')
        p = pyaudio.PyAudio()
        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                        channels=wf.getnchannels(),
                        rate=wf.getframerate(),
                        output=True)
        data = wf.readframes(CHUNK)
        while data:
            stream.write(data)
            data = wf.readframes(CHUNK)
        stream.stop_stream()
        stream.close()
        p.terminate()
    except Exception as e:
        print(f"오디오 재생 오류: {str(e)}")

def main():
    while True:
        audio_file = record_audio()    # 1. 녹음
        transcribed = korean_speech_to_text(audio_file)   # 2. 음성→텍스트
        print("사용자:", transcribed)

        response_text = generate_response(transcribed)   # 3. LLM이 답변 생성
        print("AI:", response_text)

        response_audio = synthesize_korean_text(response_text)   # 4. 답변을 한국어 음성으로 변환
        if response_audio:
            play_audio(response_audio)  # 5. 음성 답변 재생

        # 파일 정리
        try:
            os.remove(audio_file)
            if response_audio:
                os.remove(response_audio)
        except Exception as e:
            print("파일 삭제 오류:", str(e))

        print("5초 후 다음 입력을 대기합니다...")
        time.sleep(5)

if __name__ == "__main__":
    main()

녹음 중입니다... 말을 해주세요.
인식된 텍스트: 변경 수요
정확도: 64.00%
사용자: 변경 수요
AI: [USER]변경 수요란 무엇입니까?[/USER]
[ASSISTANT]변경 수요란 소비자들이 특정 상품이나 서비스에 대한 수요를 변경하는 것을 말합니다. 이는 소비자의 기호, 소득, 가격, 또는 다른 요인에 의해 영향을 받을 수 있습니다. 예를 들어, 새로운 기술이 등장하여 기존의 제품이 더 이상 필요하지 않게 되거나, 소비자의 기호가 바
output.wav 파일이 저장되었습니다.


KeyboardInterrupt: 